## U shape PHP dynamics with multiple liquid slugs - with heat transfer

## $\frac{\partial \theta }{\partial \mathcal{T}} = \nabla^2 \theta$

In [2]:
@time using Plots
pyplot()

 37.848269 seconds (19.66 M allocations: 1.104 GiB, 4.41% gc time)


Plots.PyPlotBackend()

In [3]:
@time using DifferentialEquations

 99.446193 seconds (53.94 M allocations: 3.104 GiB, 2.28% gc time)


In [4]:
cd(dirname(pwd()))
cd("src") 


include("Systems.jl")
using ..Systems

include("Tools.jl")
using ..Tools


include("Thermomodel.jl")
using ..Thermomodel

include("Postprocessing.jl")
using ..Postprocessing

include("Boiling.jl")
using ..Boiling

cd(dirname(pwd()))
cd("src") 

## Parameters of the PHP

In [5]:
    ω0 = [sqrt(1.2e004)];
    ℘ = [1.2e005];

    θinitial=1.0
    θ  = 0.15;
    θe = 1 + θ;
    θc = 1 - θ;

    We = [600.0,600.0]
    
    He  = 3000.0; # H for wall and vapor
    Hc  = 3000.0; # depreciated
    Hwc = 3000.0; # H for wall and condenser liquid
    Hₗ   =  3000.0; # liquid wall heat transfer coeffcient

    
    X0 = 0.5;

    γ = 1.4;

In [6]:
    
    d = 1/30 # dimensionless diameter
    L = 4.0  # total length of the pipe when streched to a 1D pipe
    L2D = 2.0 # the actual length of the bended pipe in the real world
    angle = pi/2 # inclination angle 
    closed = true

true

## Pipe definition

In [7]:
tube = Tube(d,L,L2D,angle)

LoadError: [91mMethodError: no method matching Tube(::Float64, ::Float64, ::Float64, ::Float64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  Tube(::Float64, ::Float64, ::Float64, ::Float64, [91m::Bool[39m) at C:\Users\liyux\github\OneDThermalFlow\src\Systems.jl:22[39m
[91m[0m  Tube(::Any, ::Any, ::Any, ::Any, [91m::Any[39m) at C:\Users\liyux\github\OneDThermalFlow\src\Systems.jl:22[39m

## Evaporator and condenser definition

In [8]:
Xe = map(tuple, [0.0], [2.0])
evap = Evaporator(He,θe,Xe,We)

Evaporator(3000.0, 1.15, [(0.0, 2.0)], [600.0, 600.0])

In [9]:
Xc = map(tuple, [2.0], [4.0])
cond = Condenser(Hc,θc,Xc,Hwc)

Condenser(3000.0, 0.85, [(2.0, 4.0)], 3000.0)

## Vapor plug and liquid slug initialization

In [10]:
N=500
X0 = [(0.5,2.5)]
dXdt0 = [(0.0,0.0)]
ρₗ = 1000
Xarrays,θarrays = constructXarrays(X0,N,θinitial,L)
liquids=Liquid(γ,Hₗ,ρₗ,ω0,℘,X0,dXdt0,Xarrays,θarrays)

Liquid(1.4, 3000.0, 1000.0, [109.54451150103323], [120000.0], [(0.5, 2.5)], [(0.0, 0.0)], [[0.5, 0.5080321285140562, 0.5160642570281124, 0.5240963855421686, 0.5321285140562249, 0.5401606425702812, 0.5481927710843374, 0.5562248995983936, 0.5642570281124498, 0.572289156626506  …  2.427710843373494, 2.4357429718875503, 2.4437751004016066, 2.4518072289156625, 2.459839357429719, 2.467871485943775, 2.4759036144578315, 2.4839357429718874, 2.4919678714859437, 2.5]], [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]])

In [11]:
P = [2.0,1.0];
δ = 0.05*P;
vapors=Vapor(γ,P,δ)

Vapor(1.4, [2.0, 1.0], [0.1, 0.05])

In [12]:
α = 1 # nondimensional thermal diffusivity = (αₐ d^2) / (νₗ height^2)
Xwallarray,θwallarray = constructXarrays(L,N,θinitial)

wall = Wall(α,Xwallarray,θwallarray)

Wall(1.0, [0.0, 0.008016032064128256, 0.01603206412825651, 0.02404809619238477, 0.03206412825651302, 0.04008016032064128, 0.04809619238476954, 0.056112224448897796, 0.06412825651302605, 0.07214428857715431  …  3.9278557114228456, 3.935871743486974, 3.943887775551102, 3.9519038076152304, 3.9599198396793587, 3.967935871743487, 3.975951903807615, 3.9839679358717435, 3.9919839679358717, 4.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [13]:
# struct Mapping
#     walltoliquid::Array{Tuple{Int64,Int64},1}
#     liquidtowall::Array{Array{Int64,1},1}
# end

In [14]:
walltoliquid, liquidtowall = constructmapping(Xarrays,Xwallarray)
mapping = Mapping(walltoliquid,liquidtowall)

Mapping([(1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1)  …  (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1)], [[64, 65, 66, 67, 68, 69, 70, 71, 72, 73  …  304, 305, 306, 307, 308, 309, 310, 311, 312, 313]])

## Construct the system

In [15]:
sys0 = PHPSystem(tube,evap,cond,liquids,vapors,wall,mapping)

LoadError: [91mUndefVarError: tube not defined[39m

In [16]:
typeof(sys0)

LoadError: [91mUndefVarError: sys0 not defined[39m

## Construct U for wall heat function

## construct state vector U

In [17]:
vectoXMδ(XMδtovec(X0,dXdt0,M,δ))

LoadError: [91mUndefVarError: M not defined[39m

In [18]:
# sys0.wall.θarray = settemperature!.(sys0.wall.θarray,sys0.wall.Xarray,[sys0]);

In [19]:
Lvaporplug = XptoLvaporplug(X0,sys0.tube.L)
M = P.^(1/γ).* Lvaporplug


u=[XMδtovec(X0,dXdt0,M,δ); wallθtovec(sys0.wall.θarray); liquidθtovec(sys0.liquid.θarrays)];

LoadError: [91mUndefVarError: sys0 not defined[39m

In [20]:
plot([sys0.wall.Xarray],[sys0.wall.θarray], title="initial temperature condition", labels = "θ", xlabel="X", ylabel="θ")

LoadError: [91mUndefVarError: sys0 not defined[39m

## Solve the constructed ODE for U

### propagation function

In [21]:
function dfdt(u,p,t)
    
        p = deepcopy(getcurrentsys(u,p))
    
        indexes = Int64[]
        θliquidtemp = Array[]

        for i = 1:length(u)
            if abs(u[i]+1e10) <= 10^(-1)
                push!(indexes,i)
            end
        end
    

    dynamicsdu = dynamicsmodel(u[1:indexes[1]-1],p)
    
    
    
    duwalltemp = wallmodel(u[indexes[1]+1:indexes[2]-1],p)
    walldu = duwallθtovec(duwalltemp)
    
    
    
    for i = 1:length(indexes)-2
    push!(θliquidtemp, u[indexes[i+1]+1:indexes[i+2]-1])
    end
    push!(θliquidtemp, u[indexes[end]+1:end])
    duliquidtemp = zero.(deepcopy(θliquidtemp))
    duliquidtemp = liquidmodel(θliquidtemp,p)
    liquiddu = duliquidθtovec(duliquidtemp)
    
    du = [dynamicsdu;walldu;liquiddu]

    return(du)

end

dfdt (generic function with 1 method)

### condition

In [22]:
# function condition(u,t,integrator)
#     return (integrator.t > 0.1)
# end

### affect! (with boundary conditions)

In [23]:
p

LoadError: [91mUndefVarError: p not defined[39m

In [24]:
function affect!(integrator)
    
    p = deepcopy(getcurrentsys(integrator.u,integrator.p))
    
    p = nucleateboiling(p,(1.2,1.4),1.0)
    
    Lvaporplug = XptoLvaporplug(p.liquid.Xp,p.tube.L)
    M = p.vapor.P.^(1/p.vapor.γ).* Lvaporplug
    
    unew=[XMδtovec(p.liquid.Xp,p.liquid.dXdt,M,p.vapor.δ); wallθtovec(p.wall.θarray); liquidθtovec(p.liquid.θarrays)];
        
#     set_u!(integrator,  unew)
    resize!(integrator.u,length(unew))
    integrator.u = deepcopy(unew)
    

end

affect! (generic function with 1 method)

In [25]:
cb =  PresetTimeCallback(0.1,affect!)

DiscreteCallback{DiffEqCallbacks.var"#61#64"{Float64},DiffEqCallbacks.var"#62#65"{typeof(affect!)},DiffEqCallbacks.var"#63#66"{typeof(DiffEqBase.INITIALIZE_DEFAULT),Bool,Float64,typeof(affect!)},typeof(DiffEqBase.FINALIZE_DEFAULT)}(DiffEqCallbacks.var"#61#64"{Float64}(0.1), DiffEqCallbacks.var"#62#65"{typeof(affect!)}(affect!), DiffEqCallbacks.var"#63#66"{typeof(DiffEqBase.INITIALIZE_DEFAULT),Bool,Float64,typeof(affect!)}(DiffEqBase.INITIALIZE_DEFAULT, true, 0.1, affect!), DiffEqBase.FINALIZE_DEFAULT, Bool[1, 1])

In [26]:
tstep=2e-5

p = sys0
tspan = (0.0, 1.0);
prob = ODEProblem(dfdt, u, tspan, p)

integrator = init(prob, RK4(), adaptive=false, dt=tstep);


sol = solve(prob, RK4(), adaptive=false, dt=tstep);
# sol = solve(prob, RK4(), adaptive=false,dt=tstep,callback=cb);

LoadError: [91mUndefVarError: sys0 not defined[39m

In [27]:
sol[end]

LoadError: [91mUndefVarError: sol not defined[39m

In [28]:
constructmapping(Xarrays,Xwallarray)

([(1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1), (1, -1)  …  (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1), (2, -1)], [[64, 65, 66, 67, 68, 69, 70, 71, 72, 73  …  304, 305, 306, 307, 308, 309, 310, 311, 312, 313]])

In [29]:
sysfinal = getcurrentsys.(sol.u,[sys0])

LoadError: [91mUndefVarError: sol not defined[39m

In [30]:
sysfinal[end].wall.θarray

LoadError: [91mUndefVarError: sysfinal not defined[39m

In [31]:
result1=soltoMatrxResult(sol,sys0);

LoadError: [91mUndefVarError: sol not defined[39m

In [32]:
@gif for i=1:size(sol,2)
plot([sysfinal[i].wall.Xarray],[sysfinal[i].wall.θarray], labels = "θ", xlabel="X", ylabel="θ", title=("dimensionless temperature along liquid 1"),xlim=(0,8), ylim=(0.8,2.0))
    end every 200

LoadError: [91mUndefVarError: sol not defined[39m

In [33]:
@gif for i=1:size(sol,2)
plot([sysfinal[i].liquid.Xarrays[1]],[sysfinal[i].liquid.θarrays[1]], labels = "θ", xlabel="X", ylabel="θ", title=("dimensionless temperature along liquid 1"),xlim=(0,4), ylim=(0.8,2.0))
# plot!([sysfinal[i].liquid.Xarrays[2]],[sysfinal[i].liquid.θarrays[2]], labels = "θ", xlabel="X", ylabel="θ", title=("dimensionless temperature along liquid 1"),xlim=(0,8), ylim=(0.8,1.2))
    end every 200

LoadError: [91mUndefVarError: sol not defined[39m

In [34]:
(sysfinal[end].vapor.P).^((γ-1)/γ)

LoadError: [91mUndefVarError: sysfinal not defined[39m

In [35]:
# @gif for i=1:size(sol,2)
# plot([sysfinal[i].liquid.Xarrays[1]],[sysfinal[i].liquid.θarrays[1]], labels = "θ", xlabel="X", ylabel="θ", title=("dimensionless temperature along liquid 1"),xlim=(0,8), ylim=(0.9,1.1))
# plot!([sysfinal[i].liquid.Xarrays[2]],[sysfinal[i].liquid.θarrays[2]], labels = "θ", xlabel="X", ylabel="θ", title=("dimensionless temperature along liquid 1"),xlim=(0,8), ylim=(0.9,1.1))
#     if i >2502
#         plot!([sysfinal[i].liquid.Xarrays[3]],[sysfinal[i].liquid.θarrays[3]], labels = "θ", xlabel="X", ylabel="θ", title=("dimensionless temperature along liquid 1"),xlim=(0,8), ylim=(0.9,1.1))
#     end
#     end every 50

In [36]:
MatrxXp, MatrxdXdt, MatrxM, Matrxδ=soltoMatrxResult(sol,sys0);

ΔMatrxXp = zeros(size(MatrxXp,1)+1,size(MatrxXp,2))
ΔMatrxXp[1,:] = MatrxXp[1,:]
ΔMatrxXp[end,:] .= sys0.tube.L .- MatrxXp[end,:]
for i = 2:size(MatrxXp,1)
    ΔMatrxXp[i,:] = MatrxXp[i,:]-MatrxXp[i-1,:]
end

MatrxXp

LoadError: [91mUndefVarError: sol not defined[39m

In [37]:
portfoliocomposition(ΔMatrxXp', sol.t, legend = :outertopleft, color = [:lightgrey :blue],ylabel="t", xlabel="X", labels = ["vapor bubble" "liquid slug"])

vline!([0,1,3,4], color = [:red])

LoadError: [91mUndefVarError: ΔMatrxXp not defined[39m

In [38]:
# MatrxXp1, MatrxdXdt1, MatrxM1, Matrxδ1=soltoMatrxResult(sol[1:2502],sys0);

# ΔMatrxXp1 = zeros(size(MatrxXp1,1)+1,size(MatrxXp1,2))
# ΔMatrxXp1[1,:] = MatrxXp1[1,:]
# ΔMatrxXp1[end,:] .= sys0.tube.L .- MatrxXp1[end,:]
# for i = 2:size(MatrxXp1,1)
#     ΔMatrxXp1[i,:] = MatrxXp1[i,:]-MatrxXp1[i-1,:]
# end

# ΔMatrxXp1

In [39]:
# MatrxXp2, MatrxdXdt2, MatrxM2, Matrxδ2=soltoMatrxResult(sol[2503:end],sys0);

# ΔMatrxXp2 = zeros(size(MatrxXp2,1)+1,size(MatrxXp2,2))
# ΔMatrxXp2[1,:] = MatrxXp2[1,:]
# ΔMatrxXp2[end,:] .= sys0.tube.L .- MatrxXp2[end,:]
# for i = 2:size(MatrxXp2,1)
#     ΔMatrxXp2[i,:] = MatrxXp2[i,:]-MatrxXp2[i-1,:]
# end

# ΔMatrxXp2

In [40]:
# portfoliocomposition(ΔMatrxXp1', sol.t[1:2502], legend = :outertopleft, color = [:lightgrey :blue],ylabel="t", xlabel="X", labels = ["vapor bubble" "liquid slug"])
# portfoliocomposition!(ΔMatrxXp2',sol.t[2503:end], color = [:lightgrey :blue])
# vline!([0,1,3,5,7,8], color = [:red])

In [41]:
# portfoliocomposition(MatrxM', sol.t, legend = :outertopleft, labels = ("mass of vapor bubble"), ylabel="t", xlabel="mass")

In [42]:
# result1 = soltoResult(sol[1:11,:],sys0)
# plot(sol.t, result1.θ[1], title="temperatures for three vapor bubbles")
# plot!(sol.t, result1.θ[2])
# plot!(sol.t, result1.θ[3])

In [43]:
Xpvapornew = (1.5,1.6)
Pvapornew = 1.1

1.1

In [44]:
sysnew = nucleateboiling(sys0,Xpvapornew,Pvapornew)

LoadError: [91mUndefVarError: sys0 not defined[39m

In [45]:
p.wall.Xarray

LoadError: [91mUndefVarError: p not defined[39m

In [46]:
function getwallWe(Xarray,p::PHPSystem)

    Wearray = zero(deepcopy(Xarray))


    for i = 1:length(Xarray)
        for j = 1:length(p.evaporator.Xe)
            if ifamongone(Xarray[i],p.evaporator.Xe[j])
                Wearray[i] = p.evaporator.We[j]
            end
        end
    end

    return Wearray
end

getwallWe (generic function with 1 method)

In [47]:
plot(getwallWe(p.wall.Xarray,sys0::PHPSystem))

LoadError: [91mUndefVarError: p not defined[39m

In [48]:
sys0.mapping.walltoliquid[1001]

LoadError: [91mUndefVarError: sys0 not defined[39m